<a href="https://colab.research.google.com/github/sarveshmalagi/tensorflow_practice/blob/master/tf_classification_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [0]:
from google.colab import files
files.upload()

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
census_df = pd.read_csv('census_data.csv')

In [0]:
census_df.head(20)

** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [0]:
census_df['income_bracket'] = census_df['income_bracket'].apply(lambda x : 0 if '<=50K' in str(x) else 1)

In [0]:
census_df.head(20)

In [0]:
target = census_df['income_bracket']
del census_df['income_bracket']

### Perform a Train Test Split on the Data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(census_df, target, test_size=0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [21]:
X_train.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
20895,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,28,United-States
3384,47,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Wife,Black,Female,15024,0,40,United-States
1832,46,Local-gov,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,24,United-States
18919,46,State-gov,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,48,United-States
31685,60,Private,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States


In [33]:
len(X_train['native_country'].unique())

42

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [0]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=9)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=16)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size=7)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=15)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=6)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=5)
gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size=2)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size=42)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [0]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [0]:
feat_cols = [workclass, education, marital_status, occupation, relationship, race, gender, native_country, age, education_num, capital_gain, capital_loss, hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=Y_train,
                                                batch_size=10, num_epochs=1000,
                                                shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [40]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplfxyd511', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d5f2360f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [0]:
model.train(input_fn=input_func, steps=5000)

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [0]:
pred_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                   batch_size=10,
                                                   num_epochs=1,
                                                   shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [54]:
predictions = list(model.predict(pred_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmplfxyd511/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [55]:
predictions[:5]

[{'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.11884972], dtype=float32),
  'logits': array([-2.0033684], dtype=float32),
  'probabilities': array([0.8811503 , 0.11884972], dtype=float32)},
 {'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.00643086], dtype=float32),
  'logits': array([-5.0401974], dtype=float32),
  'probabilities': array([0.99356914, 0.00643085], dtype=float32)},
 {'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([0.16252816], dtype=float32),
  'logits': array([-1.6395364], dtype=float32),
  'probabilities': array([0.83747184, 0.16252816], dtype=float32)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [0]:
pred_classes = [np.argmax(prediction['probabilities']) for prediction in predictions]

In [61]:
len(list(pred_classes))

9769

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [0]:
from sklearn.metrics import classification_report

In [66]:
print(classification_report(Y_test, pred_classes, target_names=['<=50k', '>50k']))

              precision    recall  f1-score   support

       <=50k       0.85      0.94      0.89      7436
        >50k       0.72      0.46      0.56      2333

    accuracy                           0.83      9769
   macro avg       0.78      0.70      0.73      9769
weighted avg       0.82      0.83      0.81      9769

